In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression
#parse data 
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
#label encoding on categorical data 

#FAMA 49CRSP Common Stocks 
df = pd.read_csv('../ee6d2f60cdafb550.csv')
df = df.dropna()

In [25]:
import sklearn.preprocessing 
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split

#preprocessing here
#sort by date 
df = df.sort_values(by = 'public_date', ascending = True)

#encode integer categories into numbers 
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df.FFI49_desc)

#todo: https://www.gsb.stanford.edu/library/articles/databases/links/financial-ratios-suite?fbclid=IwAR0EGNGk9DdxQjEHfdaoUhdY3tNzAWDogYDzuuJi1zT_muL-uJtWQw19Fzk

#get output first 
ewlabels = df.indret_ew
vwlabels = df.indret_vw

#3year on year change as a prediction feature, raw pct change 
yoythree = ewlabels.diff(periods = 3)
#3 years rolling percent change, averaged ie. (y1-y2 + (y3-y2)change)/2 
rollavgpct = ewlabels.rolling(3).mean()

#drop first 3 years
df = df.iloc[3:]
ewlabels = ewlabels.iloc[3:]
yoythree = yoythree.iloc[3:]
#yoypctthree = yoypctthree.iloc[3:]
rollavgpct = rollavgpct.iloc[3:]

#add -1 and 1 so the bins will take on bins to be equal and set to max -1 and 1
extrema = pd.Series([-1,1])
ewnlabels = ewlabels.append(extrema)

#make a new output (bucket by percentage?)
enc = KBinsDiscretizer(n_bins=8, encode='ordinal',strategy = 'uniform')
ewnlabels = np.asarray(ewnlabels)
ewnlabels = ewnlabels.reshape((-1,1))
labels_binned = enc.fit_transform(ewnlabels)

labels_binned = labels_binned[:-2]

#1 Split-Timer series data, 0.64 Train, 0.16 dev, 0.2 Test
x_train, x_test, y_train, y_test = train_test_split(df, labels_binned, test_size = 0.2, shuffle = False)
x_tra, x_dev, y_tra, y_dev = train_test_split(x_train, y_train, test_size = 0.2, shuffle = False)

       public_date FFI49_desc  NFIRM  indret_ew  indret_vw  dpr_Median  \
23102     20130430      HARDW     10  -0.035516  -0.036117       0.029   
23121     20130430      STEEL      4  -0.073378  -0.052634       0.683   
23111     20130430        OIL     27  -0.029983  -0.008038       0.170   
23126     20130430      WHLSL      8   0.018041   0.018382       0.301   
23096     20130430      ELCEQ      2  -0.012313  -0.009147       0.463   
23084     20130430      BANKS     28   0.004377   0.019437       0.232   
23083     20130430      AUTOS      5   0.017207   0.031931       0.051   
23087     20130430      BOOKS      1  -0.078189  -0.078189       0.437   
23081     20130430       AERO      5  -0.017740   0.006943       0.267   
23085     20130430       BEER      5   0.027749   0.038080       0.526   
23120     20130430      SOFTW     26  -0.010368   0.021435       0.000   
23082     20130430      AGRIC      1   0.014816   0.014816       0.287   
23089     20130430      BUSSV     16  